# Fry Counter Grid Search Contours (FC_GSC_BestFit)

In [1]:
import cv2 as cv
import numpy as np
import os
import pandas as pd

In [2]:
# Function for opening image
def get_img(img_path, grayscale=True):
    img = cv.imread(img_path)
    if grayscale:
        return cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    return img

In [3]:
# Function for cropping image
def crop_img(img):
    img_centerX = img.shape[1] // 2
    img_centerY = img.shape[0] // 2

    # Initialize mask to use
    mask = np.zeros(img.shape, dtype=np.uint8)

    # Crop the image
    region_to_cut = cv.ellipse(mask, (img_centerX, img_centerY), (547, 547), 0, 0, 360, 255, -1)
    cropped_img = cv.bitwise_or(img, img, mask=region_to_cut)
    return cropped_img[:, img_centerX-600:img_centerX+600]

In [4]:
# Gaussian Blur Function
def gaussian_blur(img, kernel_a, kernel_b):
    return cv.GaussianBlur(img, (kernel_a, kernel_b), 0)

In [5]:
# Remove the background from the fry image
def img_diff(img, bg_img):
    return cv.absdiff(img, bg_img)

In [6]:
# Image Segmentation Using Thresholding Function
def threshold_img(diff_img, type='OTSU', block=0, c=0):
    if type == 'OTSU':
        _, thresh_img = cv.threshold(diff_img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        return thresh_img
    else:
        return cv.adaptiveThreshold(diff_img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_OTSU, block, c)

In [7]:
# Histogram Counter
def histogram_counter(thres_img, pixel_to_count='white'):
    if pixel_to_count == 'white':
        return np.count_nonzero(thres_img == 255)
    else:
        return np.count_nonzero(thres_img == 0)

In [8]:
# Countour Detection
def contour_detection(thres_img):
    contours, _ = cv.findContours(thres_img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    return contours

## Finding the best fit for the Fry Counter Grid Search Contours

In [9]:
# Initialize Best Kernels Based on Grid Searches
kernel_img_100, kernel_bg_100 = (9, 9), (5, 13)
kernel_img_200, kernel_bg_200 = (13, 5), (7, 3)
kernel_img_300, kernel_bg_300 = (5, 5), (5, 11)
kernel_img_400, kernel_bg_400 = (11, 11), (9, 3)

In [10]:
# Path to the images (100)
path = '../IMAGES/100/'
list_images_100 = [f for f in os.listdir(path) if f.startswith('my_photo')]
list_paths_100 = [path + f for f in list_images_100]

# Path to the images (200)
path = '../IMAGES/200/'
list_images_200 = [f for f in os.listdir(path) if f.startswith('my_photo')]
list_paths_200 = [path + f for f in list_images_200]


# Path to the images (300)
path = '../IMAGES/300/'
list_images_300 = [f for f in os.listdir(path) if f.startswith('my_photo')]
list_paths_300 = [path + f for f in list_images_300]


# Path to the images (400)
path = '../IMAGES/400/'
list_images_400 = [f for f in os.listdir(path) if f.startswith('my_photo')]
list_paths_400 = [path + f for f in list_images_400]

In [11]:
# Testing for kernel 100
df_100 = pd.DataFrame(columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])

for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_100:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_100[0], kernel_img_100[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_100[0], kernel_bg_100[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_100 = np.abs(np.mean(contours_val) - 100)
    accuracy = ((100 - diff_to_100) / 100 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_100 = pd.concat([df_100, pd.DataFrame([[100, 100, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])

for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_200:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_100[0], kernel_img_100[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_100[0], kernel_bg_100[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_200 = np.abs(np.mean(contours_val) - 200)
    accuracy = ((200 - diff_to_200) / 200 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_100 = pd.concat([df_100, pd.DataFrame([[100, 200, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])


for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_300:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_100[0], kernel_img_100[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_100[0], kernel_bg_100[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_300 = np.abs(np.mean(contours_val) - 300)
    accuracy = ((300 - diff_to_300) / 300 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_100 = pd.concat([df_100, pd.DataFrame([[100, 300, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])


for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_400:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_100[0], kernel_img_100[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_100[0], kernel_bg_100[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_400 = np.abs(np.mean(contours_val) - 400)
    accuracy = ((400 - diff_to_400) / 400 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_100 = pd.concat([df_100, pd.DataFrame([[100, 400, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])


df_100

,Kernel Used,Actual Count,BG Used,Accuracy
0,100,100,bg_img100,99.980392
0,100,100,bg_img200,-337.862745
0,100,100,bg_img300,-521.666667
0,100,100,bg_img400,-163.509804
0,100,200,bg_img100,-31.981818
0,100,200,bg_img200,88.563636
0,100,200,bg_img300,33.036364
0,100,200,bg_img400,1.609091
0,100,300,bg_img100,36.460000
0,100,300,bg_img200,99.193333


In [12]:
# Testing for kernel 200
df_200 = pd.DataFrame(columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])

for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_100:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_200[0], kernel_img_200[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_200[0], kernel_bg_200[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_100 = np.abs(np.mean(contours_val) - 100)
    accuracy = ((100 - diff_to_100) / 100 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_200 = pd.concat([df_200, pd.DataFrame([[200, 100, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])

for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_200:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_200[0], kernel_img_200[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_200[0], kernel_bg_200[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_200 = np.abs(np.mean(contours_val) - 200)
    accuracy = ((200 - diff_to_200) / 200 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_200 = pd.concat([df_200, pd.DataFrame([[200, 200, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])


for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_300:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_200[0], kernel_img_200[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_200[0], kernel_bg_200[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_300 = np.abs(np.mean(contours_val) - 300)
    accuracy = ((300 - diff_to_300) / 300 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_200 = pd.concat([df_200, pd.DataFrame([[200, 300, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])


for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_400:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_200[0], kernel_img_200[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_200[0], kernel_bg_200[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_400 = np.abs(np.mean(contours_val) - 400)
    accuracy = ((400 - diff_to_400) / 400 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_200 = pd.concat([df_200, pd.DataFrame([[200, 400, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])


df_200

,Kernel Used,Actual Count,BG Used,Accuracy
0,200,100,bg_img100,74.333333
0,200,100,bg_img200,-592.000000
0,200,100,bg_img300,-626.313725
0,200,100,bg_img400,-344.235294
0,200,200,bg_img100,-80.754545
0,200,200,bg_img200,99.981818
0,200,200,bg_img300,-14.981818
0,200,200,bg_img400,-31.018182
0,200,300,bg_img100,13.920000
0,200,300,bg_img200,77.733333


In [13]:
# Testing for kernel 300
df_300 = pd.DataFrame(columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])

for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_100:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_300[0], kernel_img_300[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_300[0], kernel_bg_300[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_100 = np.abs(np.mean(contours_val) - 100)
    accuracy = ((100 - diff_to_100) / 100 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_300 = pd.concat([df_300, pd.DataFrame([[300, 100, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])

for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_200:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_300[0], kernel_img_300[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_300[0], kernel_bg_300[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_200 = np.abs(np.mean(contours_val) - 200)
    accuracy = ((200 - diff_to_200) / 200 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_300 = pd.concat([df_300, pd.DataFrame([[300, 200, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])


for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_300:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_300[0], kernel_img_300[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_300[0], kernel_bg_300[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_300 = np.abs(np.mean(contours_val) - 300)
    accuracy = ((300 - diff_to_300) / 300 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_300 = pd.concat([df_300, pd.DataFrame([[300, 300, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])


for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_400:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_300[0], kernel_img_300[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_300[0], kernel_bg_300[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_400 = np.abs(np.mean(contours_val) - 400)
    accuracy = ((400 - diff_to_400) / 400 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_300 = pd.concat([df_300, pd.DataFrame([[300, 400, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])


df_300

,Kernel Used,Actual Count,BG Used,Accuracy
0,300,100,bg_img100,35.392157
0,300,100,bg_img200,-440.176471
0,300,100,bg_img300,-624.450980
0,300,100,bg_img400,-236.176471
0,300,200,bg_img100,-84.745455
0,300,200,bg_img200,90.818182
0,300,200,bg_img300,-19.545455
0,300,200,bg_img400,-20.863636
0,300,300,bg_img100,12.440000
0,300,300,bg_img200,87.413333


In [14]:
# Testing for kernel 400
df_400 = pd.DataFrame(columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])

for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_100:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_400[0], kernel_img_400[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_400[0], kernel_bg_400[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_100 = np.abs(np.mean(contours_val) - 100)
    accuracy = ((100 - diff_to_100) / 100 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_400 = pd.concat([df_400, pd.DataFrame([[400, 100, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])

for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_200:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_400[0], kernel_img_400[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_400[0], kernel_bg_400[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_200 = np.abs(np.mean(contours_val) - 200)
    accuracy = ((200 - diff_to_200) / 200 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_400 = pd.concat([df_400, pd.DataFrame([[400, 200, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])


for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_300:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_400[0], kernel_img_400[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_400[0], kernel_bg_400[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_300 = np.abs(np.mean(contours_val) - 300)
    accuracy = ((300 - diff_to_300) / 300 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_400 = pd.concat([df_400, pd.DataFrame([[400, 300, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])


for bg_num in [100, 200, 300, 400]:
    contours_val = np.array([])
    for img_path in list_paths_400:
        img = get_img(img_path)
        bg_img = get_img(f'../IMAGES/{bg_num}/background.jpg')
        # Crop the image
        img = crop_img(img)
        bg_img = crop_img(bg_img)
        # Gaussian Blur
        img = gaussian_blur(img, kernel_img_400[0], kernel_img_400[1])
        bg_img = gaussian_blur(bg_img, kernel_bg_400[0], kernel_bg_400[1])
        # Image Difference
        diff_img = img_diff(img, bg_img)
        # OTSU Thresholding
        thres_img = threshold_img(diff_img)
        # Contour Detection
        contours = contour_detection(thres_img)
        contours_val = np.append(contours_val, len(contours))
    # Accuracy measures how close is the mean contours to 100
    diff_to_400 = np.abs(np.mean(contours_val) - 400)
    accuracy = ((400 - diff_to_400) / 400 ) * 100
    # pd concat to add the accuracy to the dataframe
    df_400 = pd.concat([df_400, pd.DataFrame([[400, 400, f'bg_img{bg_num}', accuracy]], columns=['Kernel Used', 'Actual Count', 'BG Used', 'Accuracy'])])


df_400

,Kernel Used,Actual Count,BG Used,Accuracy
0,400,100,bg_img100,-60.176471
0,400,100,bg_img200,-441.117647
0,400,100,bg_img300,-705.803922
0,400,100,bg_img400,-329.196078
0,400,200,bg_img100,-118.136364
0,400,200,bg_img200,61.718182
0,400,200,bg_img300,-52.018182
0,400,200,bg_img400,-64.490909
0,400,300,bg_img100,-17.666667
0,400,300,bg_img200,73.733333


In [15]:
# Combine all dataframes
df = pd.concat([df_100, df_200, df_300, df_400])
df = df[df['Accuracy'] >= 90].sort_values(by='Accuracy', ascending=False).reset_index(drop=True)
df

,Kernel Used,Actual Count,BG Used,Accuracy
0,200,200,bg_img200,99.981818
1,100,100,bg_img100,99.980392
2,300,300,bg_img300,99.953333
3,400,400,bg_img400,99.935000
4,100,300,bg_img200,99.193333
5,100,400,bg_img100,96.495000
6,400,300,bg_img300,91.226667
7,300,200,bg_img200,90.818182
